In [1]:
import sys
sys.path.insert(0, '../..')

In [45]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from config import RANDOM_STATE, N_SPLITS
from evaluate import get_pred_labels
from helper import _process_pred_labels

In [30]:
NAME = 'keras'
oof_pred_proba = pd.read_csv(f'../../oof_predictions/{NAME}/{NAME}_pred_proba.csv', index_col='review_id')
oof_pred_labels = pd.read_csv(f'../../oof_predictions/{NAME}/{NAME}_pred_labels.csv', index_col='review_id')
test_pred_proba = pd.read_csv(f'../../test_predictions/{NAME}/{NAME}_pred_proba.csv', index_col='review_id')
test_pred_labels = pd.read_csv(f'../../test_predictions/{NAME}/{NAME}_pred_labels.csv', index_col='review_id')
target = pd.read_pickle('../../data/prepared/target.pkl')

In [10]:
oof_pred_labels.head()

,0,1,2,3,4,5,6,7,8
review_id,,,,,,,,,
0,1,0,1,0,1,0,0,0,0
1,0,0,1,0,1,0,0,0,1
2,1,0,1,0,1,0,0,0,0
5,1,0,1,0,1,0,0,0,0
7,0,0,1,0,1,0,0,0,1


In [31]:
def get_tresholds(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> pd.DataFrame:
    y_pred_labels = pd.DataFrame(np.zeros_like(y_true, dtype=np.int8), index=y_true.index, columns=y_true.columns)
    best_tresholds = {}
    for col in y_true.columns:
        candidates = [0.0] + y_pred[col].round(2).sort_values().unique().tolist() + [1.0]
        scores = [f1_score(y_true[col].values, np.where(y_pred[col].values >= tresh, 1, 0))
                  for tresh in candidates]
        idx = np.argmax(scores)
        best_score = scores[idx]
        best_treshold = candidates[idx]
        best_tresholds[col] = best_treshold
        y_pred_labels.loc[:, col] = np.where(y_pred[col] >= best_treshold, 1, 0)
        print(f'col {col}, best_score {best_score}, best_treshold {best_treshold}')
    
    print('f1 samples', f1_score(y_true, y_pred_labels, average='samples', zero_division=0))
    return best_tresholds

In [19]:
def get_tresholds2(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> pd.DataFrame:
    candidates = np.unique(oof_pred_proba.round(3).values)
    scores = [f1_score(y_true.values, np.where(y_pred.values >= tresh, 1, 0), average='samples', zero_division=0)
              for tresh in candidates]
    idx = np.argmax(scores)
    best_score = scores[idx]
    best_treshold = candidates[idx]
    print(f'best_score {best_score}, best_treshold {best_treshold}')

    return best_treshold

In [32]:
best_treshold = get_tresholds2(target, oof_pred_proba)

best_score 0.8394803050554289, best_treshold 0.311


In [33]:
best_tresholds = get_tresholds(target, oof_pred_proba)

col 0, best_score 0.8609808827458888, best_treshold 0.46
col 1, best_score 0.5058885108610311, best_treshold 0.23
col 2, best_score 0.005813953488372093, best_treshold 0.01
col 3, best_score 0.715919085312225, best_treshold 0.3
col 4, best_score 0.05242463958060289, best_treshold 0.02
col 5, best_score 0.29652042360060515, best_treshold 0.11
col 6, best_score 0.32757557063541026, best_treshold 0.11
col 7, best_score 0.35225618631732164, best_treshold 0.12
col 8, best_score 0.8666397145418433, best_treshold 0.45
f1 samples 0.8285118376831562


In [26]:
f1_score(target, get_pred_labels(oof_pred_proba.values), average='samples', zero_division=0)

0.8325586049866653

In [29]:
oof_pred_proba.agg(['min', 'max']).round(4)

,0,1,2,3,4,5,6,7,8
min,0.0,0.0000,0.0000,0.0000,0.00,0.0000,0.0000,0.0000,0.0000
max,1.0,0.9768,0.0349,0.9942,0.16,0.9879,0.8916,0.9772,0.9993


In [ ]:
oof_pred_proba

In [37]:
test_blind_pred_labels = get_pred_labels(test_pred_proba.values)

review_id
3       8
4       8
13      8
14      0
17    0,8
Name: target, dtype: object

In [49]:
(pd.DataFrame(
    data=test_blind_pred_labels, 
    index=test_pred_proba.index, 
    columns=test_pred_proba.columns)
.apply(_process_pred_labels, axis=1)
.rename('target')
.to_csv('keras_samll_labse_test_blind_pred_labels.csv'))

In [51]:
test_col_by_col_pred_labels = pd.DataFrame(data=np.zeros_like(test_pred_proba), columns=test_pred_proba.columns, index=test_pred_proba.index)
for col in target.columns:
    test_col_by_col_pred_labels[col] = np.where(test_pred_proba[col].values >= best_tresholds[col], 1, 0)

In [52]:
(test_col_by_col_pred_labels
.apply(_process_pred_labels, axis=1)
.rename('target')
.to_csv('keras_samll_labse_test_col_by_col_pred_labels.csv'))

In [54]:
test_one_tresh_labels = pd.DataFrame(data=np.zeros_like(test_pred_proba, dtype=np.int8), columns=test_pred_proba.columns, index=test_pred_proba.index)
test_one_tresh_labels.loc[:, :] = np.where(test_pred_proba.values >= best_treshold, 1, 0)
test_one_tresh_labels.head()

,0,1,2,3,4,5,6,7,8
review_id,,,,,,,,,
3,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,1
13,0,0,0,0,0,0,0,0,1
14,1,0,0,0,0,0,0,0,0
17,1,0,0,0,0,0,0,0,1


In [55]:
(test_one_tresh_labels
.apply(_process_pred_labels, axis=1)
.rename('target')
.to_csv('keras_samll_labse_test_one_tresh_labels.csv'))